<a href="https://colab.research.google.com/github/Imtiazul-Islam/ai-models/blob/main/Text-To-Image/z-image-turbo-on-t4-on-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎨 Z-Image-Turbo | AI Image Generator

> **High-speed text-to-image generation optimized for Google Colab T4 GPU**

---

## 📋 Overview

This notebook uses **Z-Image-Turbo FP8** — a fast, efficient diffusion model that generates high-quality images in just **4-6 steps**. Perfect for quick iterations and experimentation.

| Feature | Specification |
|---------|---------------|
| **Model** | Z-Image-Turbo FP8 (Quantized) |
| **Architecture** | Lumina2 + Qwen CLIP |
| **Optimized For** | Colab T4 GPU (Free Tier) |
| **Generation Time** | ~10-15 seconds |
| **Resolution** | Up to 2048×2048 |

---

## 🚀 How to Use

### Step 1: Run Setup Cell
Run **PART 1** to install ComfyUI and download the required models (~4GB total).
> ⏱️ First-time setup takes 3-5 minutes

### Step 2: Run Generator Cell  
Run **PART 2** to load models and launch the interactive UI.
> ⏱️ Model loading takes ~30 seconds

### Step 3: Generate Images
1. Enter your **positive prompt** (what you want)
2. Enter your **negative prompt** (what to avoid)
3. Adjust settings if needed (defaults work great!)
4. Click **🎨 Generate Image**
5. Download your image using the **📥 Download** button

---

## ⚙️ Recommended Settings

| Parameter | Value | Notes |
|-----------|-------|-------|
| **Steps** | `6` | More isn't better for Turbo models |
| **CFG Scale** | `1.2` | Keep low (1.0-1.5) for natural results |
| **Sampler** | `euler` | Fastest and cleanest |
| **Scheduler** | `simple` | Default, works best |
| **Seed** | `0` | Random each time (or set for reproducibility) |

---

## 💡 Prompting Tips

### ✅ Good Prompt Example:
```
a majestic lion in golden savanna, dramatic sunset lighting,
wildlife photography, 8k, sharp focus, National Geographic style
```

### ❌ Avoid:
- Very short prompts ("a cat")
- Conflicting descriptions
- Too many concepts in one prompt

### 📝 Prompt Formula:
```
[Subject] + [Setting/Environment] + [Lighting] + [Style] + [Quality tags]
```

---

## 📂 Output

- Generated images are saved to: `/content/ComfyUI/output/`
- Filename format: `z_turbo_YYYYMMDD_HHMMSS.png`
- Use the **Download** button for instant download

---

## ⚠️ Troubleshooting

| Issue | Solution |
|-------|----------|
| **CUDA out of memory** | Reduce resolution or restart runtime |
| **Model loading fails** | Re-run Part 1 to redownload models |
| **Black/corrupted image** | Try different seed or adjust CFG |
| **Slow generation** | Reduce steps to 4, use euler sampler |

---

## 📚 Learn More

- Run the **📖 Parameter Guide** cell for detailed explanations of each setting
- Experiment with different aspect ratios for various use cases
- Save seeds of images you like to recreate them later

---

<div align="center">

**Made with ❤️ for the AI Art Community**

Visit my profile here: [Imtiazul Islam's GitHub](https://github.com/Imtiazul-Islam/)

In [ ]:
# @title # **🚀 PART 1: ENVIRONMENT SETUP**
# @markdown ### ComfyUI Installation & Model Download
# @markdown This cell sets up ComfyUI and downloads the Z-Image-Turbo FP8 models optimized for Colab T4 GPU.

import os
import subprocess
from pathlib import Path

# Configuration
WORK_DIR = Path("/content/ComfyUI")
MODEL_REPO = "ImtiazulIslam/Z-Image-Turbo-FP8"

class SetupManager:
    """Manages the installation and configuration of ComfyUI environment."""

    def __init__(self, work_dir: Path):
        self.work_dir = work_dir
        self.models = {
            "diffusion": {
                "url": f"https://huggingface.co/{MODEL_REPO}/resolve/main/z-image-turbo-fp8-e4m3fn.safetensors",
                "path": work_dir / "models/diffusion_models",
                "filename": "z-image-turbo-fp8-e4m3fn.safetensors"
            },
            "clip": {
                "url": f"https://huggingface.co/{MODEL_REPO}/resolve/main/qwen_3_4b.safetensors",
                "path": work_dir / "models/clip",
                "filename": "qwen_3_4b.safetensors"
            },
            "vae": {
                "url": f"https://huggingface.co/{MODEL_REPO}/resolve/main/ae.safetensors",
                "path": work_dir / "models/vae",
                "filename": "ae.safetensors"
            }
        }

    def clone_repository(self):
        """Clone ComfyUI repository if not exists."""
        if not self.work_dir.exists():
            print("📦 Cloning ComfyUI repository...")
            subprocess.run(
                ["git", "clone", "https://github.com/Imtiazul-Islam/ComfyUI", str(self.work_dir)],
                check=True, stdout=subprocess.DEVNULL
            )
            print("✅ Repository cloned successfully")
        else:
            print("✅ ComfyUI already exists")

    def install_dependencies(self):
        """Install required Python packages and system dependencies."""
        print("📚 Installing dependencies...")
        os.chdir(self.work_dir)

        # Install Python requirements
        subprocess.run(
            ["pip", "install", "-q", "-r", "requirements.txt"],
            check=True
        )

        # Install aria2 for fast downloads
        subprocess.run(
            ["apt-get", "-y", "install", "-qq", "aria2"],
            check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL
        )
        print("✅ Dependencies installed")

    def download_model(self, model_type: str):
        """Download a specific model using aria2."""
        model_info = self.models[model_type]
        model_info["path"].mkdir(parents=True, exist_ok=True)

        output_file = model_info["path"] / model_info["filename"]

        if output_file.exists():
            print(f"⏭️  {model_type.upper()} model already exists")
            return

        print(f"⬇️  Downloading {model_type.upper()} model...")
        subprocess.run([
            "aria2c",
            "--console-log-level=error",
            "-c", "-x", "16", "-s", "16", "-k", "1M",
            model_info["url"],
            "-d", str(model_info["path"]),
            "-o", model_info["filename"]
        ], check=True)
        print(f"✅ {model_type.upper()} model downloaded")

    def setup(self):
        """Run complete setup process."""
        print("=" * 60)
        print("🎨 Z-IMAGE-TURBO SETUP")
        print("=" * 60)

        self.clone_repository()
        self.install_dependencies()

        print("\n📥 Downloading models...")
        for model_type in self.models.keys():
            self.download_model(model_type)

        print("\n" + "=" * 60)
        print("✨ Setup complete! Ready to generate images.")
        print("=" * 60)

# Execute setup
setup_manager = SetupManager(WORK_DIR)
setup_manager.setup()

In [ ]:
# @title # **🎨 PART 2: Z-IMAGE-TURBO GENERATOR**
# @markdown ### Load Models
# @markdown This cell makes the ui ready to use while loading models at VRAM.

import os
import sys
import random
import time
import datetime
from pathlib import Path
import base64
from io import BytesIO

import torch
import numpy as np
from PIL import Image
from IPython.display import display, HTML
import ipywidgets as widgets

# Setup environment
os.chdir(WORK_DIR)
sys.path.insert(0, str(WORK_DIR))

from nodes import NODE_CLASS_MAPPINGS

# ============================================================
# DARK MODE CSS
# ============================================================

display(HTML('''
<style>
    /* Force text colors for dark mode compatibility */
    .dm-text { color: #e0e0e0 !important; }
    .dm-muted { color: #a0a0a0 !important; }
    .dm-header { color: #ffffff !important; }

    /* Transparent/subtle backgrounds */
    .dm-card {
        background: rgba(255,255,255,0.05) !important;
        border: 1px solid rgba(255,255,255,0.1) !important;
        border-radius: 10px !important;
        padding: 15px !important;
        margin-bottom: 12px !important;
    }

    .dm-section-title {
        color: #60a5fa !important;
        font-weight: 600 !important;
        font-size: 1.1em !important;
        padding-bottom: 8px !important;
        margin-bottom: 12px !important;
        border-bottom: 1px solid rgba(255,255,255,0.15) !important;
    }

    /* Status messages */
    .dm-status-wait {
        background: rgba(251, 191, 36, 0.15) !important;
        border-left: 3px solid #fbbf24 !important;
        color: #fcd34d !important;
        padding: 12px 15px !important;
        border-radius: 6px !important;
        margin-bottom: 12px !important;
    }

    .dm-status-success {
        background: rgba(34, 197, 94, 0.15) !important;
        border-left: 3px solid #22c55e !important;
        color: #4ade80 !important;
        padding: 12px 15px !important;
        border-radius: 6px !important;
        margin-bottom: 12px !important;
    }

    .dm-status-error {
        background: rgba(239, 68, 68, 0.15) !important;
        border-left: 3px solid #ef4444 !important;
        color: #f87171 !important;
        padding: 12px 15px !important;
        border-radius: 6px !important;
    }

    /* Placeholder */
    .dm-placeholder {
        padding: 40px !important;
        text-align: center !important;
        color: #888 !important;
        border: 2px dashed rgba(255,255,255,0.2) !important;
        border-radius: 10px !important;
        background: rgba(255,255,255,0.02) !important;
    }

    /* Image info */
    .dm-info-grid {
        display: flex !important;
        flex-wrap: wrap !important;
        gap: 10px !important;
        margin-top: 15px !important;
    }

    .dm-info-item {
        background: rgba(255,255,255,0.08) !important;
        padding: 8px 14px !important;
        border-radius: 6px !important;
        font-size: 0.9em !important;
    }

    .dm-info-label {
        color: #888 !important;
        font-size: 0.8em !important;
    }

    .dm-info-value {
        color: #e0e0e0 !important;
        font-weight: 600 !important;
    }

    /* Download button */
    .dm-download-btn {
        display: inline-block !important;
        padding: 10px 20px !important;
        background: linear-gradient(135deg, #3b82f6, #8b5cf6) !important;
        color: white !important;
        text-decoration: none !important;
        border-radius: 8px !important;
        font-weight: 600 !important;
        margin-top: 15px !important;
        transition: all 0.2s !important;
    }

    .dm-download-btn:hover {
        transform: scale(1.03) !important;
        box-shadow: 0 4px 15px rgba(59, 130, 246, 0.4) !important;
    }

    /* Tips box */
    .dm-tips {
        background: rgba(59, 130, 246, 0.1) !important;
        border-left: 3px solid #3b82f6 !important;
        padding: 12px 15px !important;
        border-radius: 6px !important;
        margin-top: 15px !important;
    }

    .dm-tips-title {
        color: #60a5fa !important;
        font-weight: 600 !important;
        margin-bottom: 8px !important;
    }

    .dm-tips ul {
        margin: 0 !important;
        padding-left: 18px !important;
        color: #a0a0a0 !important;
    }

    .dm-tips li {
        margin-bottom: 4px !important;
    }
</style>
'''))

# ============================================================
# LOAD MODELS
# ============================================================

print("🔄 Loading models into VRAM...")

UNETLoader = NODE_CLASS_MAPPINGS["UNETLoader"]()
CLIPLoader = NODE_CLASS_MAPPINGS["CLIPLoader"]()
VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()
CLIPTextEncode = NODE_CLASS_MAPPINGS["CLIPTextEncode"]()
KSampler = NODE_CLASS_MAPPINGS["KSampler"]()
VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()
EmptyLatentImage = NODE_CLASS_MAPPINGS["EmptyLatentImage"]()

with torch.inference_mode():
    unet = UNETLoader.load_unet("z-image-turbo-fp8-e4m3fn.safetensors", "fp8_e4m3fn_fast")[0]
    clip = CLIPLoader.load_clip("qwen_3_4b.safetensors", type="lumina2")[0]
    vae = VAELoader.load_vae("ae.safetensors")[0]

print("✅ Models loaded successfully!\n")

# ============================================================
# CONFIGURATION
# ============================================================

OUTPUT_DIR = Path(WORK_DIR) / "output"
OUTPUT_DIR.mkdir(exist_ok=True)

ASPECT_RATIOS = {
    "1:1 Square (1024×1024)": (1024, 1024),
    "16:9 Landscape (1344×768)": (1344, 768),
    "9:16 Portrait (768×1344)": (768, 1344),
    "4:3 Photo (1152×896)": (1152, 896),
    "3:2 Classic (1216×832)": (1216, 832),
    "21:9 Cinematic (1536×640)": (1536, 640),
}

# ============================================================
# GENERATION FUNCTION
# ============================================================

@torch.inference_mode()
def generate_image(positive_prompt, negative_prompt, width, height,
                   steps, cfg, seed, sampler_name, scheduler, denoise):

    if seed == 0:
        seed = random.randint(0, 2**63 - 1)

    positive = CLIPTextEncode.encode(clip, positive_prompt)[0]
    negative = CLIPTextEncode.encode(clip, negative_prompt)[0]
    latent = EmptyLatentImage.generate(width, height, batch_size=1)[0]

    samples = KSampler.sample(
        unet, seed, steps, cfg, sampler_name, scheduler,
        positive, negative, latent, denoise=denoise
    )[0]

    decoded = VAEDecode.decode(vae, samples)[0].detach()
    img_array = np.array(decoded * 255, dtype=np.uint8)[0]
    image = Image.fromarray(img_array)

    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"z_turbo_{timestamp}.png"
    output_path = OUTPUT_DIR / filename
    image.save(output_path)

    return image, seed, str(output_path)

def create_download_button(image, filename):
    buffer = BytesIO()
    image.save(buffer, format='PNG')
    b64 = base64.b64encode(buffer.getvalue()).decode()
    return f'<a download="{filename}" href="data:image/png;base64,{b64}" class="dm-download-btn">📥 Download Image</a>'

# ============================================================
# WIDGETS
# ============================================================

style = {'description_width': '110px'}

# Header
header = widgets.HTML('''
<div style="text-align: center; padding: 20px 0; margin-bottom: 15px;
            border-bottom: 1px solid rgba(255,255,255,0.1);">
    <h1 class="dm-header" style="margin: 0; font-size: 1.8em;">⚡ Z-Image-Turbo Generator</h1>
    <p class="dm-muted" style="margin: 8px 0 0 0;">FP8 Optimized • 4-6 Steps • Colab T4</p>
</div>
''')

# Prompts
positive_prompt = widgets.Textarea(
    value="a majestic lion in golden savanna, dramatic sunset lighting, wildlife photography, 8k, sharp focus",
    placeholder='Describe what you want to see...',
    layout=widgets.Layout(width='100%', height='80px')
)

negative_prompt = widgets.Textarea(
    value="blurry, ugly, bad quality, distorted, watermark, low resolution",
    placeholder='Describe what to avoid...',
    layout=widgets.Layout(width='100%', height='50px')
)

# Settings
aspect_ratio = widgets.Dropdown(
    options=list(ASPECT_RATIOS.keys()),
    value="1:1 Square (1024×1024)",
    description='Aspect Ratio:',
    style=style, layout=widgets.Layout(width='100%')
)

width_input = widgets.IntText(value=1024, description='Width:', style=style, layout=widgets.Layout(width='48%'))
height_input = widgets.IntText(value=1024, description='Height:', style=style, layout=widgets.Layout(width='48%'))

steps = widgets.IntSlider(value=6, min=1, max=20, description='Steps:', style=style, layout=widgets.Layout(width='100%'))
cfg = widgets.FloatSlider(value=1.2, min=1.0, max=3.0, step=0.1, description='CFG Scale:', style=style, layout=widgets.Layout(width='100%'))
seed_input = widgets.IntText(value=0, description='Seed:', style=style, layout=widgets.Layout(width='100%'))

sampler = widgets.Dropdown(
    options=['euler', 'euler_ancestral', 'dpmpp_2m', 'dpmpp_sde'],
    value='euler', description='Sampler:', style=style, layout=widgets.Layout(width='48%')
)

scheduler = widgets.Dropdown(
    options=['simple', 'normal', 'karras', 'exponential', 'sgm_uniform'],
    value='simple', description='Scheduler:', style=style, layout=widgets.Layout(width='48%')
)

denoise = widgets.FloatSlider(value=1.0, min=0.0, max=1.0, step=0.05, description='Denoise:', style=style, layout=widgets.Layout(width='100%'))

# Button
generate_btn = widgets.Button(
    description='🎨 Generate Image',
    button_style='primary',
    layout=widgets.Layout(width='100%', height='45px')
)

# Output
output_area = widgets.Output(layout=widgets.Layout(width='100%'))

# ============================================================
# CALLBACKS
# ============================================================

def on_aspect_change(change):
    dims = ASPECT_RATIOS.get(change['new'])
    if dims:
        width_input.value, height_input.value = dims

aspect_ratio.observe(on_aspect_change, names='value')

def on_generate(btn):
    btn.disabled = True
    btn.description = '⏳ Generating...'
    output_area.clear_output(wait=True)

    with output_area:
        try:
            display(HTML('<div class="dm-status-wait">⏳ <b>Generating image...</b> Please wait.</div>'))

            start = time.time()
            image, actual_seed, path = generate_image(
                positive_prompt.value, negative_prompt.value,
                width_input.value, height_input.value,
                steps.value, cfg.value, seed_input.value,
                sampler.value, scheduler.value, denoise.value
            )
            elapsed = time.time() - start

            output_area.clear_output(wait=True)

            with output_area:
                # Success
                display(HTML(f'<div class="dm-status-success">✅ <b>Generated in {elapsed:.1f}s</b></div>'))

                # Image
                display(image)

                # Info grid
                filename = Path(path).name
                display(HTML(f'''
                    <div class="dm-info-grid">
                        <div class="dm-info-item">
                            <span class="dm-info-label">Seed</span><br>
                            <span class="dm-info-value">{actual_seed}</span>
                        </div>
                        <div class="dm-info-item">
                            <span class="dm-info-label">Resolution</span><br>
                            <span class="dm-info-value">{width_input.value}×{height_input.value}</span>
                        </div>
                        <div class="dm-info-item">
                            <span class="dm-info-label">Steps</span><br>
                            <span class="dm-info-value">{steps.value}</span>
                        </div>
                        <div class="dm-info-item">
                            <span class="dm-info-label">CFG</span><br>
                            <span class="dm-info-value">{cfg.value}</span>
                        </div>
                        <div class="dm-info-item">
                            <span class="dm-info-label">Sampler</span><br>
                            <span class="dm-info-value">{sampler.value}</span>
                        </div>
                        <div class="dm-info-item">
                            <span class="dm-info-label">Time</span><br>
                            <span class="dm-info-value">{elapsed:.1f}s</span>
                        </div>
                    </div>
                    <div style="text-align: center;">
                        {create_download_button(image, filename)}
                    </div>
                    <p class="dm-muted" style="text-align: center; font-size: 0.85em; margin-top: 10px;">
                        📁 Saved: {filename}
                    </p>
                '''))

        except Exception as e:
            output_area.clear_output(wait=True)
            with output_area:
                display(HTML(f'<div class="dm-status-error">❌ <b>Error:</b> {str(e)}</div>'))

    btn.disabled = False
    btn.description = '🎨 Generate Image'

generate_btn.on_click(on_generate)

# ============================================================
# LAYOUT
# ============================================================

def section_title(text, icon=""):
    return widgets.HTML(f'<div class="dm-section-title">{icon} {text}</div>')

# Left Panel
left_panel = widgets.VBox([
    section_title("Prompts", "📝"),
    widgets.HTML('<span class="dm-muted">Positive Prompt</span>'),
    positive_prompt,
    widgets.HTML('<span class="dm-muted" style="margin-top: 8px; display: block;">Negative Prompt</span>'),
    negative_prompt,
    widgets.HTML('<div style="height: 12px;"></div>'),
    generate_btn,
    widgets.HTML('<div style="height: 15px;"></div>'),
    section_title("Output", "🖼️"),
    output_area,
], layout=widgets.Layout(width='58%', padding='10px'))

# Right Panel
right_panel = widgets.VBox([
    section_title("Image Settings", "📐"),
    aspect_ratio,
    widgets.HBox([width_input, height_input]),
    widgets.HTML('<div style="height: 12px;"></div>'),
    section_title("Generation Settings", "⚙️"),
    steps,
    cfg,
    seed_input,
    widgets.HBox([sampler, scheduler]),
    denoise,
    widgets.HTML('''
        <div class="dm-tips">
            <div class="dm-tips-title">💡 Quick Tips</div>
            <ul>
                <li><b>Steps:</b> 4-8 optimal for Z-Turbo</li>
                <li><b>CFG:</b> 1.0-1.5 for natural results</li>
                <li><b>Seed 0:</b> Random seed each time</li>
            </ul>
        </div>
    '''),
], layout=widgets.Layout(width='40%', padding='10px'))

# Main
main_layout = widgets.HBox([left_panel, right_panel])

# Show placeholder
with output_area:
    display(HTML('''
        <div class="dm-placeholder">
            <div style="font-size: 2em; margin-bottom: 10px;">🎨</div>
            <span class="dm-muted">Your generated image will appear here</span>
        </div>
    '''))

# Display
display(header)
display(main_layout)

In [ ]:
# @title # **📖 PARAMETER GUIDE** { display-mode: "form" }
# @markdown ### All Types of Parameters
# @markdown This cell shows a guide of the parameters used in the model.

from IPython.display import display, HTML

guide_html = '''
<style>
    .guide-container { font-family: system-ui, sans-serif; max-width: 900px; }
    .guide-section {
        background: rgba(255,255,255,0.05);
        border: 1px solid rgba(255,255,255,0.1);
        border-radius: 10px;
        padding: 20px;
        margin-bottom: 20px;
    }
    .guide-title {
        color: #60a5fa;
        font-size: 1.3em;
        font-weight: 700;
        margin-bottom: 15px;
        padding-bottom: 10px;
        border-bottom: 1px solid rgba(255,255,255,0.1);
    }
    .guide-text { color: #d0d0d0; line-height: 1.7; }
    .guide-text b { color: #f0f0f0; }
    .guide-table { width: 100%; border-collapse: collapse; margin: 15px 0; }
    .guide-table th {
        background: rgba(59, 130, 246, 0.2);
        color: #60a5fa;
        padding: 10px;
        text-align: left;
        border-radius: 5px 5px 0 0;
    }
    .guide-table td {
        padding: 10px;
        border-bottom: 1px solid rgba(255,255,255,0.05);
        color: #c0c0c0;
    }
    .guide-table tr:hover td { background: rgba(255,255,255,0.03); }
    .guide-highlight {
        background: rgba(34, 197, 94, 0.15);
        color: #4ade80;
        padding: 2px 8px;
        border-radius: 4px;
        font-weight: 600;
    }
    .guide-code {
        background: rgba(255,255,255,0.1);
        padding: 2px 6px;
        border-radius: 4px;
        font-family: monospace;
        color: #fbbf24;
    }
    .guide-tip {
        background: rgba(59, 130, 246, 0.1);
        border-left: 3px solid #3b82f6;
        padding: 12px 15px;
        border-radius: 5px;
        margin-top: 15px;
        color: #93c5fd;
    }
    .guide-warning {
        background: rgba(251, 191, 36, 0.1);
        border-left: 3px solid #fbbf24;
        padding: 12px 15px;
        border-radius: 5px;
        margin-top: 15px;
        color: #fcd34d;
    }
</style>

<div class="guide-container">

    <!-- STEPS -->
    <div class="guide-section">
        <div class="guide-title">🔢 STEPS</div>
        <div class="guide-text">
            <p><b>What it does:</b> Number of iterations to refine the image from noise to final result.</p>
            <p><b>Analogy:</b> Like a sculptor chiseling a statue — more steps = more refinement.</p>

            <table class="guide-table">
                <tr><th>Steps</th><th>Quality</th><th>Speed</th><th>Use Case</th></tr>
                <tr><td>1-3</td><td>Rough, basic shapes</td><td>~3 sec</td><td>Quick previews</td></tr>
                <tr><td class="guide-highlight">4-6</td><td class="guide-highlight">✅ Optimal</td><td class="guide-highlight">~10 sec</td><td class="guide-highlight">Recommended</td></tr>
                <tr><td>8-12</td><td>Slightly more detail</td><td>~20 sec</td><td>Fine details</td></tr>
                <tr><td>15+</td><td>Diminishing returns</td><td>~30+ sec</td><td>Overkill for Turbo</td></tr>
            </table>

            <div class="guide-tip">
                💡 <b>Z-Turbo is optimized for low steps.</b> Using 6 steps gives 95% of the quality at 3x the speed.
            </div>
        </div>
    </div>

    <!-- CFG SCALE -->
    <div class="guide-section">
        <div class="guide-title">🎛️ CFG SCALE (Classifier-Free Guidance)</div>
        <div class="guide-text">
            <p><b>What it does:</b> Controls how strictly the AI follows your prompt.</p>
            <p><b>Analogy:</b> Like giving directions to an artist — low = creative freedom, high = strict rules.</p>

            <table class="guide-table">
                <tr><th>CFG</th><th>Effect</th><th>Best For</th></tr>
                <tr><td class="guide-highlight">1.0-1.5</td><td class="guide-highlight">✅ Natural, balanced</td><td class="guide-highlight">General use</td></tr>
                <tr><td>2.0-3.0</td><td>Stronger adherence</td><td>Specific prompts</td></tr>
                <tr><td>5.0+</td><td>Oversaturated, artificial</td><td>⚠️ Avoid</td></tr>
            </table>

            <div class="guide-warning">
                ⚠️ <b>High CFG = Bad for Turbo models.</b> Values above 3.0 cause oversaturated colors and artifacts.
            </div>
        </div>
    </div>

    <!-- SAMPLER -->
    <div class="guide-section">
        <div class="guide-title">🎨 SAMPLER</div>
        <div class="guide-text">
            <p><b>What it does:</b> Mathematical algorithm for removing noise step-by-step.</p>
            <p><b>Analogy:</b> Different painting techniques — each has its own style and speed.</p>

            <table class="guide-table">
                <tr><th>Sampler</th><th>Style</th><th>Speed</th><th>Best For</th></tr>
                <tr><td class="guide-highlight">euler</td><td class="guide-highlight">Clean, smooth</td><td class="guide-highlight">Fast</td><td class="guide-highlight">✅ Default choice</td></tr>
                <tr><td>euler_ancestral</td><td>More random/creative</td><td>Fast</td><td>Artistic variation</td></tr>
                <tr><td>dpmpp_2m</td><td>Balanced detail</td><td>Medium</td><td>Complex scenes</td></tr>
                <tr><td>dpmpp_sde</td><td>High quality</td><td>Slower</td><td>Maximum detail</td></tr>
            </table>
        </div>
    </div>

    <!-- SCHEDULER -->
    <div class="guide-section">
        <div class="guide-title">📅 SCHEDULER</div>
        <div class="guide-text">
            <p><b>What it does:</b> Controls the pace of noise removal across steps.</p>
            <p><b>Analogy:</b> Like cooking — some recipes need high heat first, others steady heat throughout.</p>

            <table class="guide-table">
                <tr><th>Scheduler</th><th>Behavior</th><th>Best For</th></tr>
                <tr><td class="guide-highlight">simple</td><td class="guide-highlight">Linear, predictable</td><td class="guide-highlight">✅ Z-Turbo default</td></tr>
                <tr><td>normal</td><td>Standard diffusion</td><td>General use</td></tr>
                <tr><td>karras</td><td>Smooth transitions</td><td>High-detail images</td></tr>
                <tr><td>exponential</td><td>Aggressive early</td><td>Fast convergence</td></tr>
                <tr><td>sgm_uniform</td><td>Uniform removal</td><td>Experimental</td></tr>
            </table>
        </div>
    </div>

    <!-- DENOISE -->
    <div class="guide-section">
        <div class="guide-title">🔊 DENOISE</div>
        <div class="guide-text">
            <p><b>What it does:</b> Strength of the generation process (how much the AI changes).</p>

            <table class="guide-table">
                <tr><th>Value</th><th>Effect</th><th>Use Case</th></tr>
                <tr><td class="guide-highlight">1.0</td><td class="guide-highlight">Full generation</td><td class="guide-highlight">✅ Text-to-image (default)</td></tr>
                <tr><td>0.7</td><td>Moderate changes</td><td>Image-to-image edits</td></tr>
                <tr><td>0.3</td><td>Subtle adjustments</td><td>Minor variations</td></tr>
                <tr><td>0.0</td><td>No change</td><td>Not useful</td></tr>
            </table>

            <div class="guide-tip">
                💡 <b>For text-to-image:</b> Always use <span class="guide-code">1.0</span>
            </div>
        </div>
    </div>

    <!-- SEED -->
    <div class="guide-section">
        <div class="guide-title">🎲 SEED</div>
        <div class="guide-text">
            <p><b>What it does:</b> Random number that determines the starting noise pattern.</p>
            <p><b>Same seed + same settings = same image</b> (reproducible results)</p>

            <table class="guide-table">
                <tr><th>Value</th><th>Behavior</th></tr>
                <tr><td class="guide-highlight">0</td><td class="guide-highlight">Random seed each time (recommended for exploration)</td></tr>
                <tr><td>Any number</td><td>Fixed result — use to recreate favorite images</td></tr>
            </table>

            <div class="guide-tip">
                💡 <b>Pro tip:</b> Found a great image? Save the seed to recreate it later or make variations!
            </div>
        </div>
    </div>

    <!-- RECOMMENDED SETTINGS -->
    <div class="guide-section" style="background: rgba(34, 197, 94, 0.1); border-color: rgba(34, 197, 94, 0.3);">
        <div class="guide-title" style="color: #4ade80;">✅ RECOMMENDED SETTINGS FOR Z-IMAGE-TURBO</div>
        <div class="guide-text">
            <table class="guide-table">
                <tr><th>Parameter</th><th>Optimal Value</th><th>Why</th></tr>
                <tr><td>Steps</td><td><span class="guide-code">6</span></td><td>Best speed/quality balance</td></tr>
                <tr><td>CFG Scale</td><td><span class="guide-code">1.2</span></td><td>Natural, follows prompt well</td></tr>
                <tr><td>Sampler</td><td><span class="guide-code">euler</span></td><td>Fast and clean results</td></tr>
                <tr><td>Scheduler</td><td><span class="guide-code">simple</span></td><td>Designed for Turbo models</td></tr>
                <tr><td>Denoise</td><td><span class="guide-code">1.0</span></td><td>Full generation</td></tr>
                <tr><td>Seed</td><td><span class="guide-code">0</span></td><td>Random for variety</td></tr>
            </table>
        </div>
    </div>

</div>
'''

display(HTML(guide_html))